In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder


In [ ]:
df = pd.read_csv("detect.csv")  # Replace with your file path

print(df.head())


   sport dport proto state       dur  sbytes  spkts     label
0   1390    53   udp   CON  0.001055     132      2       NaN
1  33661  1024   udp   CON  0.036133     528      4       NaN
2   1464    53   udp   CON  0.001119     146      2  Exploits
3   3593    53   udp   CON  0.001209     132      2       NaN
4  49664    53   udp   CON  0.001169     146      2       NaN


In [3]:
# Replace NaN with 0, otherwise replace with 1 in the 'label' column
df['label'] = df['label'].apply(lambda x: 0 if pd.isna(x) else 1)


In [4]:
print(df.head())

   sport dport proto state       dur  sbytes  spkts  label
0   1390    53   udp   CON  0.001055     132      2      0
1  33661  1024   udp   CON  0.036133     528      4      0
2   1464    53   udp   CON  0.001119     146      2      1
3   3593    53   udp   CON  0.001209     132      2      0
4  49664    53   udp   CON  0.001169     146      2      0


In [ ]:
le = LabelEncoder()

# Encode categorical columns
df['sport'] = le.fit_transform(df['sport'])
df['dport'] = le.fit_transform(df['dport'])
df['proto'] = le.fit_transform(df['proto'])
df['state'] = le.fit_transform(df['state'])
df['label'] = le.fit_transform(df['label'])
print(df.head())

   sport  dport  proto  state       dur  sbytes  spkts  label
0   3278  16443    120      2  0.001055     132      2      0
1  19641     86    120      2  0.036133     528      4      0
2   3893  16443    120      2  0.001119     146      2      1
3  21476  16443    120      2  0.001209     132      2      0
4  32737  16443    120      2  0.001169     146      2      0


In [6]:
# Features
X = df.drop(columns=['label'])

# Target variable
y = df['label']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
smote = SMOTE(random_state=42)

# Apply SMOTE to balance the dataset
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE, class distribution in y_train: {y_train.value_counts()}")
print(f"After SMOTE, class distribution in y_train: {y_train_res.value_counts()}")


Before SMOTE, class distribution in y_train: label
0    70888
1     9112
Name: count, dtype: int64
After SMOTE, class distribution in y_train: label
0    70888
1    70888
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training data and transform both training and test sets
X_train_res = scaler.fit_transform(X_train_res)
X_test = scaler.transform(X_test)

print(X_train_res[:5])


[[ 0.70813067 -0.45390371  0.13104754  0.01412937 -0.21034992 -0.03489687
   0.27115278]
 [ 0.84602366 -0.34742952  0.13104754  0.01412937 -0.21298536 -0.03847306
   0.19501447]
 [-0.22741439 -0.54462656  0.13104754  0.01412937  0.05536439  1.00144402
   0.86122469]
 [ 0.41522608  0.35393258  0.13104754  0.01412937  0.17417413 -0.0548308
  -0.14760793]
 [ 0.66948791  0.73098798  0.44550307  1.05259802 -0.22251756 -0.06077456
  -0.24278082]]


In [10]:
# Initialize the Random Forest Classifier
clf = RandomForestClassifier(random_state=42)

# Train the model on the resampled and scaled data
clf.fit(X_train_res, y_train_res)


RandomForestClassifier(random_state=42)

In [ ]:
# Make predictions on the test set
y_pred = clf.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     17681
           1       0.88      0.98      0.93      2319

    accuracy                           0.98     20000
   macro avg       0.94      0.98      0.96     20000
weighted avg       0.98      0.98      0.98     20000

Confusion Matrix:
[[17373   308]
 [   52  2267]]


In [ ]:
#hyper tuning, not necssary now, because it will take a lot of time :)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_res, y_train_res)

# Best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

random_search.fit(X_train, y_train)

print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

best_model_random = random_search.best_estimator_


NameError: name 'df' is not defined